# Setup and Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set
import warnings
import re
from pandas.io import gbq
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy import sparse

# Get the Data

In [3]:
chicago_data = """
                    SELECT unique_key, date, primary_type, location_description, 
                            arrest, domestic, community_area,year, latitude, longitude,
                    FROM `gdac-327115.Chicago.chicago2`
                    WHERE year >= 2011
               """
chicago_data = gbq.read_gbq(chicago_data, project_id="gdac-327115")

In [4]:
#Read in an Excel file with a one to one mapping between Chicago community areas and districts
chicago_districts = pd.read_excel("ChicagoCommunityAreas.xlsx")

In [5]:
#Data type can't be joined on an int
chicago_districts.community_area = chicago_districts["community_area"].astype("string")
chicago_data.community_area = chicago_data["community_area"].astype("string")

In [6]:
#Outer join the two data sets
chicago = chicago_data.merge(chicago_districts, how="outer", left_on="community_area", right_on="community_area", )

In [7]:
#Check data types
chicago.dtypes

unique_key                            Int64
date                    datetime64[ns, UTC]
primary_type                         object
location_description                 object
arrest                              boolean
domestic                            boolean
community_area                       string
year                                  Int64
latitude                            float64
longitude                           float64
district_name                        object
community_name                       object
dtype: object

In [8]:
#Change the object data types to categorical
chicago.loc[:, chicago.dtypes == "object"] = chicago.select_dtypes(["object"]).apply(lambda x: x.astype("category"))

In [9]:
#Drop the community area variable since we have a community name variable
chicago.drop("community_area", axis = 1, inplace = True)

### Explore first few rows

In [10]:
chicago.head()

,unique_key,date,primary_type,location_description,arrest,domestic,year,latitude,longitude,district_name,community_name
0,11228094,2017-11-20 15:00:00+00:00,DECEPTIVE PRACTICE,POOL ROOM,False,False,2017,NaN,NaN,Far North,NORTH PARK
1,12135292,2020-08-13 12:00:00+00:00,DECEPTIVE PRACTICE,BANK,False,False,2020,41.993402,-87.712280,Far North,NORTH PARK
2,8905294,2012-11-27 17:30:00+00:00,PUBLIC PEACE VIOLATION,RESTAURANT,False,False,2012,41.978265,-87.708742,Far North,NORTH PARK
3,8805120,2012-09-17 00:30:00+00:00,DECEPTIVE PRACTICE,TAXICAB,False,False,2012,41.985277,-87.713827,Far North,NORTH PARK
4,9030552,2013-02-21 00:00:00+00:00,CRIM SEXUAL ASSAULT,RESIDENCE,False,True,2013,41.994323,-87.720213,Far North,NORTH PARK


In [11]:
#Check the number of rows and columns
chicago.shape

(2971810, 11)